# How people hashtagging
### During election 2020

> A hack on [election tweets dataset](https://www.kaggle.com/manchunhui/us-election-2020-tweets), please upvote the dataset for his dedicated effort to collect the data

![data image](https://storage.googleapis.com/kaggle-datasets-images/935914/1582537/4fc3f5fa3b371aad3c097e7fb892305b/dataset-cover.jpg?t=2020-10-23-13-26-22)

The election is at some extent all over, but what worries me is the ever so separated perception of media, stories, even the basic definition of things that leads to a devided society.  As the problem described so vividly by the Netflix documentary [The Social Delimma](https://www.imdb.com/title/tt11464826/). 

This notebook tries to look closely at the diversity forms of tweet texts and how online society devides. And I hope the data science community can move toward the goal of bringing down segregation/news integrity in one day

In [ ]:
!pip install -q forgebox

In [ ]:
from forgebox.imports import *
from forgebox.df import PandasDisplay

from datetime import datetime
from typing import List, Tuple
import random

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import plotly.express as px
from wordcloud import WordCloud
from forgebox.html import DOM
from forgebox.images.widgets import image_dom
from itertools import chain

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

All the tweets are either hash tagging D.J.Trump or J.R.Biden， saved separately in 2 csv files

In [ ]:
DATA = Path("/kaggle/input/us-election-2020-tweets/")

trump_df = pd.read_csv(
    DATA/"hashtag_donaldtrump.csv",
    lineterminator='\n', error_bad_lines=False)
biden_df = pd.read_csv(DATA/"hashtag_joebiden.csv", error_bad_lines=False)
# Remove the error lines (last column nan)
trump_df = trump_df[~trump_df.collected_at.isna()]
biden_df = biden_df[~biden_df.collected_at.isna()]

Remove the error lines (last column nan)

In [ ]:
trump_df.columns

In [ ]:
us_countries = ['United States of America','United States']

def compare_vc(col, filter_query="tweet_id!=''"):
    """
    Compare value counts of 2 candidate
    """
    DOM(f"Value counts under 2 candidates on column {col.upper()}", "h3")()
    return biden_df.query(filter_query).vc(col).rename(columns={col: "biden"}).join(
        trump_df.query(filter_query).vc(col).rename(columns={col: "trump"})
    ).head(20)

In [ ]:
compare_vc("country")

In [ ]:
compare_vc("state",f"(country in {us_countries})")

In [ ]:
compare_vc("source")

But to be honest, the hashtag distribution among countries, states etc demonstrate nothing meaning full, we still don't know what kind of information is pro-trump, what is anti-trump

## Combine the tweet data into one

In [ ]:
all_df = pd.concat([trump_df, biden_df])

all_df = all_df.query("retweet_count>2").reset_index(drop=True)

def replace_us(x): return "US" if x in us_countries else x

all_df["country"]= all_df["country"].apply(replace_us)

## Find the hash_tag(#) and the at(@)

With regex ```r"@(\w+)"``` for @ and ```r"#(\w+)"``` for hash tag

In [ ]:
%%time
import re

def find_at(text):
    return re.findall(r"@(\w+)",text)

def find_hashtag(text):
    return re.findall(r"#(\w+)",text)

all_df["at_s"] = all_df.tweet.apply(find_at)
all_df["hash_tags"] = all_df.tweet.apply(find_hashtag)

## Frequent Hash Tag

In [ ]:
p_display=PandasDisplay(max_colwidth = 0,max_rows=120)

Here are the top mentioned hashtags 

In [ ]:
all_hashtags = chain(*all_df["hash_tags"])
hashtag_stat = pd.DataFrame({"hashtag":all_hashtags}).vc("hashtag")
with p_display:
    display(hashtag_stat.head(100))

# Manual review
> I mannually review 12 tweets sampled from each hashtag, of the top frequent hashtags. And put the hashtag into "Pro Trump" or "Pro Biden" list to the best of my judgement.(eg if some hashtag looks like pro-trump, but the sample tweets hashtagged this phrase isn't always pro trump, I won't label it pro trump)

> To be clear I'm a coder/ Chinese citizen who's hooked to the US 2020 election rallies news/ tweets/ posts/ conspiracy theory non-stop, all of those from both sides, so my judgement might not be very accurate, besides, there are lots of non-english expression. I seriously hoped someone can try and relabel that

In [ ]:
def start_review(hashtag_stat):
    gen = iter(hashtag_stat.index)
    def next_ht():
        """
        Review hashtags one by one
        """
        ht = next(gen)
        DOM(f"#{ht}","h3")()
        has_ht = all_df["hash_tags"].apply(lambda x: ht in x)
        total = has_ht.sum()
        DOM(f"Total count: {total}, {int(total*100/len(all_df))}%","h4")()
        frame = DOM("","div",{"class":"row"})
        for tweet in all_df[has_ht].sample(12).tweet:
            frame.append(DOM(tweet.replace(ht, f"<span class='text-danger'>{ht}</span>"),
                             "div",{"class":"col-sm-4"}))
        frame()
    return next_ht

In [ ]:
PRO_TRUMP = ["Trump2020","MAGA","HunterBiden", "China", "MAGA2020", "KAG", "Trump2020Landslide", "Trump2020LandslideVictory",
             "TRUMP2020ToSaveAmerica","maga", "HunterBidenLaptop", "BidenCrimeFamiily", "PresidentTrump",
             "ccp","hunter", "HunterBidenEmails", "4MoreYears","TrumpPence2020"
            ]
PRO_BIDEN = [ "TrumpOut", "TrumpGenocide", "Biden2020", "TrumpMeltdown","BlueWave","BidenHarris2020", "BidenHaris2020",
             "ByeByeTrump", "TrumpIsLosing","Resist", "VoteHimOut", "PresidentElectJoe", "Truth",
             "TrumpVirus","VoteBidenHarris2020", "VoteBlue","CountEveryVote", "BidenPresident",
             "BidenHarrisToSaveAmerica","TrumpOut", "TrumpCrimeFamily", "BidenHarrisLandslide2020",
             "TrumpIsANationalDisgrace","TrumpCollapse","PresidentElect", "ByeByeTrump","VoteBidenHarrisToSaveAmerica",
             "VoteBlueToSaveAmerica", "VoteBiden", "DumpTrump","JoeBiden2020"
            ]

In [ ]:
next_ht = start_review(hashtag_stat)

I repeated run next_ht to review the tweet message 😭

In [ ]:
next_ht()

## Display some samples

In [ ]:
with p_display:
    display(
        all_df[["tweet", "user_name", "retweet_count", "at_s", "hash_tags"]].sample(20))

## All ats and hashtags

## Learn the graph

In [ ]:
class HashTagGraph(Dataset):
    @classmethod
    def from_df(cls, df: pd.DataFrame):
        # count unique hash tag numbers in a row
        df["ht_count"] =  df.hash_tags.apply(lambda x:len(set(x)))
        # we use the lower case
        df["hash_tags_lower"] = df.hash_tags.apply(lambda x:list(i.lower() for i in x))
        # we use the group with more than 1 hash tag
        hashtag_groups = list(df.query("ht_count>1")["hash_tags_lower"])
        return cls(hashtag_groups)
        
    def __init__(self, hashtag_groups: List[str]):
        self.hashtag_groups = hashtag_groups
        self.all_hashtags = list(chain(*self.hashtag_groups))
        self.hashtag_stat = pd.DataFrame(
            pd.DataFrame({"hashtags": self.all_hashtags})["hashtags"].value_counts()
        )
        self.total_pool = np.arange(len(self.hashtag_stat))
        self.i2c = np.array(self.hashtag_stat.index)
        self.c2i = dict((v,k) for k,v in enumerate(self.hashtag_stat.index))
        
    def __len__(self): return len(self.hashtag_groups)
        
    def __getitem__(self, idx: int) -> Tuple[int]:
        group = self.hashtag_groups[idx]
        choices = random.sample(group, 2 )
        return tuple([
            self.c2i[choices[0]],
            self.c2i[choices[1]],
            random.choice(self.total_pool),
        ])

In [ ]:
ds = HashTagGraph.from_df(all_df)
ds[5]

In [ ]:
def get_dl(df, batch_size=256, num_workers=4):
    ds = HashTagGraph.from_df(df)
    return DataLoader(
        ds, shuffle=True,
        batch_size=batch_size,
        num_workers=num_workers)

def get_gen(df, **kwargs):
    return iter(get_dl(df, **kwargs))

In [ ]:
gen = get_gen(all_df)

for i in range(30):
    x1,x2,z = next(gen)
print(",".join(map(str,map(lambda x:x.shape, [x1, x2, z]))))

## Train a graph

In [ ]:
import pytorch_lightning as pl
pl.__version__

In [ ]:
class GraphData(pl.LightningDataModule):
    def __init__(self, df, batch_size=256):
        super().__init__()
        self.df=df
        self.batch_size=batch_size
        self.ds = HashTagGraph.from_df(self.df)
        self.c2i=ds.c2i
        self.i2c=ds.i2c
        
    def train_dataloader(self):
        return DataLoader(self.ds, shuffle=True, batch_size=self.batch_size, num_workers=4)

In [ ]:
graph_data = GraphData(all_df)
num_cates = len(graph_data.i2c)
num_cates

## Loss Function
> Cosine Embedding Loss

$\text{loss}(x, y) =
    \begin{cases}
    1 - \cos(x_1, x_2), & \text{if } y = 1 \\
    \max(0, \cos(x_1, x_2) - \text{margin}), & \text{if } y = -1
    \end{cases}
$

In [ ]:
class Graph(pl.LightningModule):
    def __init__(
        self,
        num_cates: int=num_cates,
        margin: float=32.
    ):
        super().__init__()
        self.num_cates = num_cates
        self.emb = nn.Embedding(num_cates, 32)
        self.crit = nn.CosineEmbeddingLoss(margin)
        
    def forward(
        self, x1, x2, z):
        x_1 = torch.cat([x1,x1])
        x_2 = torch.cat([x2,z])
        y = torch.cat([
            torch.ones_like(x2),
            -torch.zeros_like(z),
        ])
        return self.crit(self.emb(x_1), self.emb(x_2), y)
        
    def configure_optimizers(self): return torch.optim.Adam(self.parameters())
        
    def training_step(self, batch, batch_idx):
        x1, x2, z = batch
        loss = self(x1, x2, z)
        return {"loss":loss}
    
graph = Graph()

In [ ]:
trainer = pl.Trainer(max_epochs=40)

In [ ]:
trainer.fit(graph, graph_data)

In [ ]:
vecs = graph.emb.weight.data.numpy()
vecs.shape

## Find closest hashtag

In [ ]:
class CosineSim:
    def __init__(self, names: List[str], vecs: np.array, top_k: int=20):
        self.names = np.array(names)
        self.n2i = dict((v,k) for k,v in enumerate(self.names))
        self.top_k = top_k
        self.vecs = vecs
        self.l2 = np.sqrt(np.power(vecs,2).sum(-1))
        self.vecs_normed = vecs/(self.l2[:,None])
        
    def __repr__(self):return f"Cosine Similarity Engine:\n\t"+",".join(self.names[:10])+",..."
    
    def __call__(self, name):
        i = self.n2i.get(name)
        
        if i is None: raise ValueError(f"[ERROR] name '{name}' not found")
        
        sim_map = (self.vecs_normed[i,None]*self.vecs_normed).sum(-1)
        df = pd.DataFrame({"names":self.names,"sim":sim_map}).sort_values(by = "sim", ascending=False)
        return df.head(self.top_k)

In [ ]:
CosineSim(graph_data.ds.hashtag_stat.index, vecs)("trump2020")

In [ ]:
top_hashtags = graph_data.ds.hashtag_stat.head(150)
top_ids = np.array(list(graph_data.ds.c2i[i] for i in top_hashtags.index))

tsne=TSNE(2, metric="cosine", )
emb_coords = tsne.fit_transform(graph.emb.weight.data.numpy()[top_ids])

# pca=PCA(2)
# emb_coords = pca.fit_transform(graph.emb.weight.data.numpy()[top_ids])

# Visualize manifold

> We visualize the the TSNE of the top 150 mentioned hashtags, and marked the blue or red for clear preference of such hashtag

> You can see the clustering effect of the graph isn't that impressive 🥶, but some similar words are groupped up together

In [ ]:
pro_trump = list(map(lambda x:x.lower(),PRO_TRUMP))
pro_biden = list(map(lambda x:x.lower(),PRO_BIDEN))

color_list =list((
    "1 pro_trump" if i in pro_trump else 
    ("0 pro_biden" if i in pro_biden else
     "2 no_preference")) for i in top_hashtags.index)
fig = px.scatter(
    emb_coords,
    x=0, y=1,
    hover_name=top_hashtags.index,
    color=color_list,
    color_discrete_map={"0 pro_biden":"blue","1 pro_trump":"red" }
)
fig.update_traces(marker_size=8)
fig.show()

## Words Ownership

> Okay let's try words ownership by country, states, some words just appearing a lot, like trump biden, but they are in almost evey tweet, but some words are only owned by certain category of data

In [ ]:
hashtags = list(chain(*all_df["hash_tags_lower"]))
hashtag_stat = pd.DataFrame({"hashtags": hashtags}).vc("hashtags")

h2count = dict(zip(hashtag_stat.index, hashtag_stat.hashtags))

In [ ]:
def calc_owner(df, col):
    keys = list(df.vc(col).index)[:20]
    result = []
    for k in keys:
        tags = list(chain(*df.query(f"{col}=='{k}'").hash_tags_lower))
        k_df = pd.DataFrame({"ht": tags}).vc("ht").reset_index()
        k_df[f"{col}"] = k
        k_df = k_df.rename(columns={"index": "hashtag", "ht":"ct"})
        result.append(k_df)
    return_df = pd.concat(result)[[col, "hashtag", "ct", ]]
    return_df["total"] = return_df["hashtag"].apply(lambda x:h2count[x])
    return_df["ownership"] = return_df["ct"] / return_df["total"]
    return return_df

def clean_onwership(df, col):
    col_df = calc_owner(df, col)
    rt = []
    wc_painter = WordCloud(
            background_color="white",
            height=500, width=800
            )
    for k in col_df[col].unique():
        k_df = col_df.query(f"{col}=='{k}'").query(f"ct>3")
        k_df = k_df.sort_values(by = ["ownership","total"], ascending=False).reset_index(drop=2).head(50)
        wc = wc_painter.generate(" ".join(k_df.hashtag))
        DOM(f"{col}:{k}", "h3")()
        image_dom(wc.to_image())()

# Word cloud hashtag by country

In [ ]:
clean_onwership(all_df, "country")

# Word cloud hashtag by state

> See what's on eash states' mind

In [ ]:
clean_onwership(all_df.query("country=='US'"), "state")